In [1]:
# Import the Packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
# Load the dataset
data = pd.read_csv('D:\GenAI\Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Pre-process the data
## Drop the irrelevant columns
data.columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1)


In [6]:
# Now, Gender and Geography are categorical variables
# Encode the categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
# One Hot Encoder for Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [10]:
# Combine
data = pd.concat([data.drop('Geography',axis = 1),geo],axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
# Save the file encoders and scaler as pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [15]:
# Train, Test and Split
X = data.drop ('Exited',axis = 1)
y = data['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [16]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [17]:
# ANN Implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

import datetime

In [18]:
# Build our ANN Model
model = Sequential ( [
    Dense (64, activation = 'relu',input_shape=(X_train.shape[1],)), # First Hidden layer
    Dense (32, activation = 'relu'),
    Dense (1, activation = 'sigmoid')])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the Model
#opt = tf.keras.optimizers.Adam(learning_rate = 0.01)

model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy']) # For multi-classification,it can be sparse-crossentropy


In [47]:
# Setup the Tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d- %H%M%S")

tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)

In [50]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(
    monitor = "val_loss",
    patience = 5,
    restore_best_weights = True
)

In [51]:
# Train the Model
history = model.fit(
    X_train,y_train,validation_data=[X_test,y_test],epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 114815475712.0000 - accuracy: 0.7945 - val_loss: 108140199936.0000 - val_accuracy: 0.8035
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 109860323328.0000 - accuracy: 0.7945 - val_loss: 103450222592.0000 - val_accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 105055444992.0000 - accuracy: 0.7945 - val_loss: 98894381056.0000 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 100396761088.0000 - accuracy: 0.7945 - val_loss: 94463860736.0000 - val_accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 95894667264.0000 - accuracy: 0.7945 - val_loss: 90210295808.0000 - val_accuracy: 0.8035
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 91535147008.0000 - accuracy: 0.7945 - val_loss: 86091112448.0000 - val_accuracy: 0.8035
Epoch 7/100
250/

In [52]:
model.save('model.h5')

d:\GenAI\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
